L'objectif est de simplement scraper et stocker sous forme de fichiers xml les différents plans de comptes

Entrée : rien

Sortie : Fichiers xml stockés dans le dossier "plan_compte"

Stratégie naïve : 

- A partir de l'url mère, on va demander au plan de comptes d'aller chercher, pour commencer, dans l'année 2020. 
- Le script va aller chercher dans l'arborescence cette année les différentes nomenclature. 
- Dans chacune de ces nomenclatures, le script va récupérer le plan de compte xml lié
- Le script va renommer le fichier par sa nomenclature
- Le fichier renommer va être localement stocké dans un dossier "plans_de_comptes/2020/..."

Questionnements, limite et autres propositions : 
- Partant du principe que l'on aurait pas besoin de le découper pour airflow (script peu utilisé), 
je me suis permi de ne pas beaucoup le découper en fonctions (mais ce serait rapide à faire)
- Stratégie plutôt globale, vu qu'elle consiste à faire une immense liste de tout les planDeCompte du site, on aurait pu imaginer un script qui va individuellement chercher les urls et téléchargé l'url dès qu'elle finie par planDeCompte.xml 
- Ce serait très rapide de faire une variante permettant de selectionner une seule année ou une seule nomenclature, c'est simplement du troncage sur le tronc du code

In [2]:
import requests
from bs4 import BeautifulSoup
import os 


def url_to_soup(url) : 
 requete = requests.get(url)
 requete_soupe = BeautifulSoup(requete.text, 'html.parser')
 return requete_soupe


ce qu'on souhaite, en soi c'est reconstituer les différents url, donc commencons au plus simple, comme si on était déjà à l'étape "année/nomenclature" avec l'exemple de M14

In [3]:
url_exemple_m14 = "http://odm-budgetaire.org/composants/normes/2020/M14"
soupe_m14 = url_to_soup(url_exemple_m14)
soupe_m14 = soupe_m14.find_all('a', string=lambda texte: 'M' in texte)
liste_new_url = []
for nom in soupe_m14 :
 liste_new_url.append(f"{url_exemple_m14}/{nom.text}planDeCompte.xml")
liste_new_url

['http://odm-budgetaire.org/composants/normes/2020/M14/M14_CCAS_INF3500/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2020/M14/M14_CCAS_SUP3500/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2020/M14/M14_CE/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2020/M14/M14_COM_500_3500/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2020/M14/M14_COM_INF500/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2020/M14/M14_COM_SUP3500/planDeCompte.xml']

Les planDeCompte prendront toujours la même forme, donc on a pas besoin de les chercher, on les connait déjà ! 
On a accès à plusieurs planDeCompte, maintenant il faut généraliser cette méthode : 

In [14]:
url_basique = 'http://odm-budgetaire.org/composants/normes/'

soupe_mere = url_to_soup(url_basique)
soupe_annees = soupe_mere.find_all('a', string=lambda texte: '20' in texte)

url_annees = []
for i in soupe_annees :
    url_annees.append(f'{url_basique}{i.text}')

annee_sans_plan_de_compte = ['http://odm-budgetaire.org/composants/normes/2010/', 'http://odm-budgetaire.org/composants/normes/2011/','http://odm-budgetaire.org/composants/normes/2012/']
url_annees_propre = [year for year in url_annees if year not in annee_sans_plan_de_compte]
url_annees_propre
 

['http://odm-budgetaire.org/composants/normes/2013/',
 'http://odm-budgetaire.org/composants/normes/2014/',
 'http://odm-budgetaire.org/composants/normes/2015/',
 'http://odm-budgetaire.org/composants/normes/2016/',
 'http://odm-budgetaire.org/composants/normes/2017/',
 'http://odm-budgetaire.org/composants/normes/2018/',
 'http://odm-budgetaire.org/composants/normes/2019/',
 'http://odm-budgetaire.org/composants/normes/2020/',
 'http://odm-budgetaire.org/composants/normes/2021/',
 'http://odm-budgetaire.org/composants/normes/2022/',
 'http://odm-budgetaire.org/composants/normes/2023/',
 'http://odm-budgetaire.org/composants/normes/2024/']

Les années 2010, 2011 et 2011 ne comportent pas de planDeCompte, ils nous sont donc inutiles, à partir de cette liste d'années on va pouvoir appliquer à chaque année la même méthode

In [5]:
liste_url_xml = []
liste_url_nomenclature = []
for i in url_annees_propre : 
 sous_soupe = url_to_soup(i)
 ensemble_nomenclatures = sous_soupe.find_all('a', string=lambda texte: 'M' in texte)
 for nomenclature in ensemble_nomenclatures : 
  liste_url_nomenclature.append(f'{i}{nomenclature.text}')

liste_url_nomenclature


['http://odm-budgetaire.org/composants/normes/2013/M14/',
 'http://odm-budgetaire.org/composants/normes/2013/M4/',
 'http://odm-budgetaire.org/composants/normes/2013/M52/',
 'http://odm-budgetaire.org/composants/normes/2013/M61/',
 'http://odm-budgetaire.org/composants/normes/2013/M71/',
 'http://odm-budgetaire.org/composants/normes/2014/M14/',
 'http://odm-budgetaire.org/composants/normes/2014/M4/',
 'http://odm-budgetaire.org/composants/normes/2014/M52/',
 'http://odm-budgetaire.org/composants/normes/2014/M61/',
 'http://odm-budgetaire.org/composants/normes/2014/M71/',
 'http://odm-budgetaire.org/composants/normes/2015/M14/',
 'http://odm-budgetaire.org/composants/normes/2015/M4/',
 'http://odm-budgetaire.org/composants/normes/2015/M52/',
 'http://odm-budgetaire.org/composants/normes/2015/M57/',
 'http://odm-budgetaire.org/composants/normes/2015/M61/',
 'http://odm-budgetaire.org/composants/normes/2015/M71/',
 'http://odm-budgetaire.org/composants/normes/2016/M14/',
 'http://odm-budg

Et on execute le même principe pour trouver les secondes parties de nomenclatures : C'est évidemment long de façon exponentiel, heureusement on a pas trop de descendants 

In [6]:

for nomen in liste_url_nomenclature : 
 soup_nom = url_to_soup(nomen)
 soup_nom = soup_nom.find_all('a', string=lambda texte: 'M' in texte)
 for dernier_enfant in soup_nom :
  liste_url_xml.append(f'{nomen}{dernier_enfant.text}planDeCompte.xml')
 

liste_url_xml
 

['http://odm-budgetaire.org/composants/normes/2013/M14/M14_CCAS_INF3500/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M14/M14_CCAS_SUP3500/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M14/M14_CE/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M14/M14_COM_500_3500/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M14/M14_COM_INF500/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M14/M14_COM_SUP3500/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M4/M4/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M4/M41/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M4/M42/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M4/M43_A/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M4/M43_D/planDeCompte.xml',
 'http://odm-budgetaire.org/composants/normes/2013/M4/M44/planDeCompte.xml',
 '

Maintenant que nous avons les url, il va falloir créer un endroit où les accueillir et les télécharger au bon endroit 

In [8]:

def creation_chemin_temp(url_xml) : 
 ''' Permet de créer les dossiers liés à l'url '''
 annee_xml_test = url_xml.split('/')[5]
 nomenclature_xml_test = url_xml.split('/')[6]
 sous_nom_text = url_xml.split('/')[7]
 chemin_fchier = f'/stockage_plan_de_compte/{annee_xml_test}/{nomenclature_xml_test}/{sous_nom_text}.xml'
 return chemin_fchier

url_test_creation_chemin = liste_url_xml[1]
chemin_test_crea = creation_chemin_temp(url_test_creation_chemin)
chemin_test_crea

'/stockage_plan_de_compte/2013/M14/M14_CCAS_SUP3500.xml'

Avec ça, on va créer les dossiers stockage_plan_de_compte, 2013 etc etc etc 

In [9]:
def _creation_des_dossiers(nom_chemin) : 
 segments = nom_chemin.split("/")
 chemin_base = "."
 for segment in segments[:-1]:  # Exclure le nom du fichier à la fin
    chemin_base = os.path.join(chemin_base, segment)
    if not os.path.exists(chemin_base):
        os.mkdir(chemin_base)


La fonction a été découpée pour faciliter sa comprehension, là voici en l'état : 

In [64]:
def creation_chemin(url_xml) : 
 ''' Permet de créer les dossiers liés à l'url '''
 annee_xml_test = url_xml.split('/')[5]
 nomenclature_xml_test = url_xml.split('/')[6]
 sous_nom_text = url_xml.split('/')[7]
 chemin_fichier = f'./stockage_plan_de_compte/{annee_xml_test}/{nomenclature_xml_test}-{sous_nom_text}.xml'
 _creation_des_dossiers(chemin_fichier)
 return chemin_fichier

Maintenant, le telechargement, quelque soit la fonction du chemin choisi : 

In [12]:
def telechargement_planDeCompte(url_du_xml, chemin_fichier) : 
 requete = requests.get(url_du_xml)
 with open(chemin_fichier, 'w', encoding='utf-8') as dl_local : 
  dl_local.write(requete.text)

La méthode complète nous donne ceci : (à ne pas lancer si on souhaite pas dl les plan de compte)

In [17]:
def scraping_plan_de_compte() : 

  url_basique = 'http://odm-budgetaire.org/composants/normes/'
  soupe_mere = url_to_soup(url_basique)
  soupe_annees = soupe_mere.find_all('a', string=lambda texte: '20' in texte)
  url_annees = []

  for i in soupe_annees :
      url_annees.append(f'{url_basique}{i.text}')
  annee_sans_plan_de_compte = ['http://odm-budgetaire.org/composants/normes/2010/',
              'http://odm-budgetaire.org/composants/normes/2011/',
              'http://odm-budgetaire.org/composants/normes/2012/']

  url_annees_propre = [year for year in url_annees if year not in annee_sans_plan_de_compte]
  #Nous avons la liste des dossier avec les années 

  liste_url_xml = []
  liste_url_nomenclature = []
  for i in url_annees_propre : 
  sous_soupe = url_to_soup(i)
  ensemble_nomenclatures = sous_soupe.find_all('a', string=lambda texte: 'M' in texte)
  for nomenclature in ensemble_nomenclatures : 
    liste_url_nomenclature.append(f'{i}{nomenclature.text}')
  #Nous avons maintenant la liste de la première partie des nomenclature : le premier M14 de M14-M14

  for nomen in liste_url_nomenclature : 
  soup_nom = url_to_soup(nomen)
  soup_nom = soup_nom.find_all('a', string=lambda texte: 'M' in texte)
  for dernier_enfant in soup_nom :
    liste_url_xml.append(f'{nomen}{dernier_enfant.text}planDeCompte.xml')
  #Nous avons maintenant la liste entière des xml plan de compte de 2013 à 2024 ! 

  for i in liste_url_xml :
  chemin_pour_dl = creation_chemin(i)
  telechargement_planDeCompte(i, chemin_pour_dl)
  #Nous avons maintenant les dossiers sous forme de : 
  #./stockage_plan_de_compte/année/

  #Remplissons les. 

  for i in liste_url_xml :
  chemin_pour_dl = creation_chemin(i)
  telechargement_planDeCompte(i, chemin_pour_dl)

  #Et voilà. 

Durée : 18.9 sec 